# Hillfort detection with LiDAR data
## Data management

## Table of contents

[Code](#code)

1. [**Initializing and training the model**](#initializing-and-training-the-model)
2. [**Evaluating the model**](#evaluating-the-model)
3. [**Hyperparameter tuning**](#hyperparameter-tuning)
4. [**Results**](#results)

[End](#end)

## Code

### Defined functions

In [52]:
# Imports
# Imports
import os
# import re
# import csv
# import typing
# import itertools
# import json
import logging
import zipfile
# import warnings
# import evaluate
# import types
import pandas as pd
import sklearn as sk
import numpy as np
import torch
# import math
import shapely
import matplotlib.pyplot as plt
import laspy # Reading LAS file format
from tqdm import tqdm # Loading bars
import geopandas as gpd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

In [18]:
class Tnet(nn.Module):
   def __init__(self, k=3):
      super().__init__()
      self.k=k
      self.conv1 = nn.Conv1d(k,64,1)
      self.conv2 = nn.Conv1d(64,128,1)
      self.conv3 = nn.Conv1d(128,1024,1)
      self.fc1 = nn.Linear(1024,512)
      self.fc2 = nn.Linear(512,256)
      self.fc3 = nn.Linear(256,k*k)

      self.bn1 = nn.BatchNorm1d(64)
      self.bn2 = nn.BatchNorm1d(128)
      self.bn3 = nn.BatchNorm1d(1024)
      self.bn4 = nn.BatchNorm1d(512)
      self.bn5 = nn.BatchNorm1d(256)
       

   def forward(self, input):
      # input.shape == (bs,n,3)
      bs = input.size(0)
      xb = F.relu(self.bn1(self.conv1(input)))
      xb = F.relu(self.bn2(self.conv2(xb)))
      xb = F.relu(self.bn3(self.conv3(xb)))
      pool = nn.MaxPool1d(xb.size(-1))(xb)
      flat = nn.Flatten(1)(pool)
      xb = F.relu(self.bn4(self.fc1(flat)))
      xb = F.relu(self.bn5(self.fc2(xb)))
      
      #initialize as identity
      init = torch.eye(self.k, requires_grad=True).repeat(bs,1,1)
      if xb.is_cuda:
        init=init.cuda()
      matrix = self.fc3(xb).view(-1,self.k,self.k) + init
      return matrix


class Transform(nn.Module):
   def __init__(self):
        super().__init__()
        self.input_transform = Tnet(k=3)
        self.feature_transform = Tnet(k=64)
        self.conv1 = nn.Conv1d(3,64,1)

        self.conv2 = nn.Conv1d(64,128,1)
        self.conv3 = nn.Conv1d(128,1024,1)
       

        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(1024)
       
   def forward(self, input):
        matrix3x3 = self.input_transform(input)
        # batch matrix multiplication
        xb = torch.bmm(torch.transpose(input,1,2), matrix3x3).transpose(1,2)

        xb = F.relu(self.bn1(self.conv1(xb)))

        matrix64x64 = self.feature_transform(xb)
        xb = torch.bmm(torch.transpose(xb,1,2), matrix64x64).transpose(1,2)

        xb = F.relu(self.bn2(self.conv2(xb)))
        xb = self.bn3(self.conv3(xb))
        xb = nn.MaxPool1d(xb.size(-1))(xb)
        output = nn.Flatten(1)(xb)
        return output, matrix3x3, matrix64x64

class PointNet(nn.Module):
    def __init__(self, classes = 10):
        super().__init__()
        self.transform = Transform()
        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, classes)
        

        self.bn1 = nn.BatchNorm1d(512)
        self.bn2 = nn.BatchNorm1d(256)
        self.dropout = nn.Dropout(p=0.3)
        self.logsoftmax = nn.LogSoftmax(dim=1)

    def forward(self, input):
        xb, matrix3x3, matrix64x64 = self.transform(input)
        xb = F.relu(self.bn1(self.fc1(xb)))
        xb = F.relu(self.bn2(self.dropout(self.fc2(xb))))
        output = self.fc3(xb)
        return self.logsoftmax(output), matrix3x3, matrix64x64

In [85]:
class PointCloudDataset(Dataset):
    def __init__(self, xyz, labels, num_points=1024):
        self.xyz = xyz
        self.labels = labels
        self.num_points = num_points

    def __len__(self):
        return len(self.xyz)  # Number of point clouds in the dataset

    def __getitem__(self, idx):
        points = self.xyz[idx]  # Points for the idx-th sample
        labels = self.labels[idx]  # Labels for the same sample
        
        # Ensure points is a 2D tensor with shape (N, 3)
        if points.ndimension() == 1:
            points = points.view(-1, 3)

        # Padding to ensure every point cloud has num_points
        if points.shape[0] < self.num_points:
            padding = torch.zeros(self.num_points - points.shape[0], 3)  # Padding with zeros
            points = torch.cat([points, padding], dim=0)  # Concatenate the points with the padding
        else:
            points = points[:self.num_points]  # Truncate if there are more than num_points

        return {'pointcloud': points, 'category': labels}


In [146]:
def pointnetloss(outputs, labels, m3x3, m64x64, criterion, m3x3_weight=0.0001, m64x64_weight=0.0001):
    if not criterion:
        criterion = torch.nn.NLLLoss()
    bs = outputs.size(0)

    # Identity matrices for regularization (to penalize transformation deviations)
    id3x3 = torch.eye(3, requires_grad=True).repeat(bs, 1, 1).to(outputs.device)
    id64x64 = torch.eye(64, requires_grad=True).repeat(bs, 1, 1).to(outputs.device)

    # Regularization terms
    diff3x3 = id3x3 - torch.bmm(m3x3, m3x3.transpose(1, 2))
    diff64x64 = id64x64 - torch.bmm(m64x64, m64x64.transpose(1, 2))

    # Compute the base loss (negative log likelihood)
    base_loss = criterion(outputs, labels)

    # Add regularization (transformation matrices)
    reg_loss = m3x3_weight * (torch.norm(diff3x3) + torch.norm(diff64x64)) / float(bs)

    return base_loss + reg_loss

In [147]:
def train(pointnet, criterion, train_loader, device, val_loader=None, epochs=15, save=True):
    optimizer = torch.optim.Adam(pointnet.parameters(), lr=0.001)
    for epoch in range(epochs): 
        pointnet.train()
        running_loss = 0.0
        for i, data in enumerate(train_loader, 0):
            inputs, labels = data['pointcloud'].to(device).float(), data['category'].to(device)
            optimizer.zero_grad()
            outputs, m3x3, m64x64 = pointnet(inputs.transpose(1,2))

            loss = pointnetloss(outputs, labels, m3x3, m64x64, criterion)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            if i % 10 == 9:    # print every 10 mini-batches
                    print('[Epoch: %d, Batch: %4d / %4d], loss: %.3f' %
                        (epoch + 1, i + 1, len(train_loader), running_loss / 10))
                    running_loss = 0.0

        pointnet.eval()
        correct = total = 0

        # validation
        if val_loader:
            with torch.no_grad():
                for data in val_loader:
                    inputs, labels = data['pointcloud'].to(device).float(), data['category'].to(device)
                    outputs, __, __ = pointnet(inputs.transpose(1,2))
                    _, predicted = torch.max(outputs.data, 1)
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()
            val_acc = 100. * correct / total
            print('Valid accuracy: %d %%' % val_acc)

        # save the model
        if save:
            torch.save(pointnet.state_dict(), "save_"+str(epoch)+".pth")


In [21]:
test_las = laspy.read('../data/classified_lazFiles/397689_2023_tava.laz')

In [34]:
# Load the CSV
normalized_polygons = pd.read_csv('../data/normalized_polygons.csv')

# Convert the list of WKT strings back to shapely Polygons
normalized_polygons['polygons'] = normalized_polygons['polygons_wkt'].apply(lambda wkt_list: [shapely.wkt.loads(wkt_str) for wkt_str in eval(wkt_list)])
normalized_polygons = normalized_polygons.drop('polygons_wkt', axis=1)

In [35]:
display(normalized_polygons)
test_poly = normalized_polygons['polygons'][0]
display(test_poly)
print((test_las.header.x_min, test_las.header.y_min, test_las.header.z_min), (test_las.header.x_max, test_las.header.y_max, test_las.header.z_max))
print(test_poly)

,laz_file,polygons
0,397689_2023_tava.laz,[POLYGON ((0.2154821638250724 0.53317514155060...
1,402642_2023_tava.laz,[POLYGON ((0.559215598856099 0.702996839769184...
2,402674_2023_tava.laz,[POLYGON ((0.7066870752023533 0.31486295908689...
3,407676_2023_tava.laz,[POLYGON ((0.6727867362787947 0.95630937255918...
4,407700_2023_tava.laz,[POLYGON ((0.119291202398017 0.732177131809294...
...,...,...
126,596650_2022_tava.laz,[POLYGON ((0.9488594959257171 0.22939210943877...
127,596651_2022_tava.laz,[POLYGON ((-0.0511505041504279 0.2293921094387...
128,598586_2022_tava.laz,[POLYGON ((0.2016820210264996 0.15814139693975...
129,600638_2022_tava.laz,[POLYGON ((0.8100281070219353 0.38243363983929...


[<POLYGON ((0.215 0.533, 0.316 0.483, 0.343 0.417, 0.345 0.322, 0.314 0.254, ...>]

(0.0, 0.0, 0.0) (1.0, 1.0, 1.0)
[<POLYGON ((0.215 0.533, 0.316 0.483, 0.343 0.417, 0.345 0.322, 0.314 0.254, ...>]


In [137]:
xyz = test_las.xyz
xyzc = np.hstack((xyz, test_las.points.array['classification'].reshape(-1, 1)))
X = xyz
y = (xyz[:, -1] == 12).astype(int)

print(X.shape, y.shape)

(4715859, 3) (4715859,)


In [138]:
class_weights = sk.utils.class_weight.compute_class_weight(class_weight='balanced', classes=np.unique(y), y=y)
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float32).cuda()  # If using GPU
criterion = torch.nn.NLLLoss(weight=class_weights_tensor)

In [139]:
X = torch.from_numpy(X)
y = torch.from_numpy(y).long()

In [140]:
train_dataset = PointCloudDataset(X, y)

In [141]:
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)

### Initializing and training the model

In [142]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [143]:
pointnet = PointNet()
pointnet.to(device);

In [ ]:
train(pointnet, criterion, train_loader, device, None, 10, False)

### Evaluating the model

### Hyperparameter tuning

### Results

## End